<a href="https://colab.research.google.com/github/tutsilianna/Gift_Recommendation_System/blob/main/Gift_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аннотация

Список литературы:
1. https://education.yandex.ru/handbook/ml/article/intro-recsys
2. Анализ текстовых данных https://habr.com/ru/companies/otus/articles/774498/
---
https://neurohive.io/ru/tutorial/sozdaem-rekomendatelnuju-sistemu-s-ispolzovaniem-biblioteki-fastai/

https://habr.com/ru/companies/skillfactory/articles/585182/   

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B

https://vc.ru/u/1389654-machine-learning/592278-5-luchshih-generatorov-sinteticheskih-dannyh-na-python-i-kak-ih-ispolzovat-kogda-vam-ne-hvataet-dannyh

### Есть три варианта реализации:


1.  Content-Based Filtering (Фильтрация на основе контента)

    **Принцип:** Анализирует характеристики подарков и предпочтения пользователя (пол, возраст, интересы, хобби).
    
    **Данные:** Описания подарков, профили пользователей.
    
    **Алгоритмы:**TF-IDF, cosine similarity.
    
    **Пример:** Если пользователь увлекается фотографией, система порекомендует фотоаппарат, объектив или аксессуары для фотосъемки.

2. Collaborative Filtering (Коллаборативная фильтрация):

    **Принцип:** Анализирует историю покупок и оценок пользователей, чтобы найти похожих пользователей и рекомендовать подарки, которые понравились им.

    **Данные:** История покупок, рейтинги, отзывы.

    **Алгоритмы:** K-Nearest Neighbors, Matrix Factorization.

    **Пример:** Если пользователь А и пользователь Б имеют похожую историю покупок, и пользователь А купил книгу, которая понравилась пользователю Б, система порекомендует эту книгу другим пользователям с похожими предпочтениями.



3. Hybrid Approach (Гибридный подход):

    **Принцип:** Комбинирует content-based и collaborative filtering для повышения точности рекомендаций.

    **Пример:** Система использует content-based filtering, чтобы найти подарки, соответствующие интересам пользователя, а затем использует collaborative filtering, чтобы ранжировать эти подарки на основе предпочтений похожих пользователей.



* Дополнительные факторы:

    **Бюджет пользователя:** Система должна учитывать бюджет пользователя при рекомендации подарков.

    **Повод:** Подарки на день рождения, Новый год или свадьбу будут отличаться.

    **Социальная информация:** Информация о друзьях и семье пользователя в социальных сетях может помочь в подборе подарков.

**Входные параметры:** (опционально) пол, возраст, интересы/хобби - строковое описание увлечений человека.

**Данные для обучения:** даные о пользователе (входные параметры) + какие подарки он выбрал. Эти данные будут синтетическими

## Генерация анкет

In [ ]:
import random

# Список интересов
interests_list = ["спорт", "музыка", "кино", "книги", "игры", "кулинария",
                  "путешествия", "искусство", "технологии", "мода", "красота",
                  "шоппинг", "автомобили", "рыбалка", "охота", "финансы",
                  "образование", "здоровье", "саморазвитие", "домашние животные",
                  "садоводство", "фотография", "танцы", "йога", "волонтерство",
                  "рисование", "живопись", "скульптура", "фотография", "музыка",
                  "танцы", "театр", "кино", "писательство", "поэзия", "рукоделие",
                  "дизайн", "кулинария", "марки", "монеты", "комиксы", "антиквариат",
                  "игрушки", "видеоигры", "компьютерные игры", "мобильные игры",
                  "настольные игры", "азартные игры"]

# Функция для генерации данных пользователя
def generate_user():
    gender = random.choice(["male", "female"])
    age = random.randint(18, 75)
    num_interests = random.randint(2, 5)  # Количество интересов
    interests = random.sample(interests_list, num_interests)
    return f"{gender}\t{age}\t{','.join(interests)}"

# Генерация и запись данных
with open("users.txt", "w") as f:
    for _ in range(5000):
        user_data = generate_user()
        f.write(user_data + "\n")

print("Файл users.txt создан!")

Файл users.txt создан!


## Подключение репозитория

In [1]:
! git clone https://github.com/tutsilianna/Gift_Recommendation_System

Cloning into 'Gift_Recommendation_System'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 49 (delta 15), reused 24 (delta 5), pack-reused 0
Receiving objects: 100% (49/49), 15.57 MiB | 15.40 MiB/s, done.
Resolving deltas: 100% (15/15), done.


## Импорт необходимых библиотек

In [2]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk import stem
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer



nltk.download("stopwords")

mystem = Mystem()
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


## Чтение данных и предобработка

In [15]:
data = pd.read_csv('/content/Gift_Recommendation_System/data/raw/ozon_data_.csv')
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,категория,подкатегория,подподкатегория,ссылка
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,Артикул: 1320591182,3.3,13,808,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-dlya-smar...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,Артикул: 619939085,4.4,205,300,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-90411567-...


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22541 entries, 0 to 22589
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            22541 non-null  object 
 1   описание            22541 non-null  object 
 2   код товара          22541 non-null  object 
 3   оценка              22541 non-null  float64
 4   количество отзывов  22541 non-null  int64  
 5   цена                22541 non-null  int64  
 6   категория           22541 non-null  object 
 7   подкатегория        22541 non-null  object 
 8   подподкатегория     22541 non-null  object 
 9   ссылка              22541 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 1.9+ MB


## **1. Подготовка данных:**



### **Предобработка данных:** обработка пропущенных значений, удаление дубликатов, преобразование столбца "код товара";    


In [17]:
data['подподкатегория'] = data['подподкатегория'].fillna('')
data['код товара'] = data['код товара'].fillna('Артикул: 0')
data['описание'] = data['описание'].fillna('')

In [18]:
data.drop_duplicates(inplace=True)

In [20]:
# удаление слова из "Код товара", удаление строк,
# где нет атрибута и преобразование столбца к численному типу

data['код товара'] = data['код товара'].str.rsplit().str.get(-1)
data = data.loc[data['код товара'].str.isdigit()]
data['код товара'] = data['код товара'].astype(np.int64)

<ipython-input-20-8f791f2d9fe1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['код товара'] = data['код товара'].astype(np.int64)


In [22]:
data.drop_duplicates(subset=['код товара'], keep='first', inplace=True)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18427 entries, 0 to 22589
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            18427 non-null  object 
 1   описание            18427 non-null  object 
 2   код товара          18427 non-null  int64  
 3   оценка              18427 non-null  float64
 4   количество отзывов  18427 non-null  int64  
 5   цена                18427 non-null  int64  
 6   ссылка              18427 non-null  object 
 7   ключевые слова      18427 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.3+ MB


In [24]:
data['ключевые слова'] = data['категория'] + ', ' + data['подкатегория'] + ', ' + data['подкатегория']
data.drop(columns=['категория', 'подкатегория', 'подподкатегория'], axis=1, inplace=True)

In [25]:
text_columns = [column for column in data.columns if data[column].dtype == 'object' and column != 'ссылка']
text_columns

['название', 'описание', 'ключевые слова']

In [112]:
# {
#     "интерес": ['подподкатегория']
# }

test = {
  "спорт": ["Cпортивные игры", "Красота, здоровье и спорт"],

  "музыка": ["Музыка", "Ударные и перкуссия", "Струнные инструменты",
             "Гитары и аксессуары", "Клавишные инструменты"],

  "кино": ["Кино", "Проекторы", "Электронные и аудиокниги", "Цифровые товары"],

  "книги": ["Книги", "Художественная литература", "Нехудожественная литература",
            "Научно-популярная литература", "Учебная литература", "Кулинарные книги",
            "Путешествия. Путеводители", "Фантастика", "Молодежная и подростковая литература",
            "Ужасы и триллеры", "Комиксы", "Юридическая литература", "Боевики",
            "Электронные и аудиокниги", "История", "Журналы и газеты",
            "Религиозные книги", "Медицина", "Приключения", "Инвестиции и недвижимость",
            "Познавательная литература", "Пьесы и драматургия", "Энциклопедии и справочники",
            "Юмор", "Античная литература", "Детям и родителям", "Фольклор", "Поэзия",
            "Открытки", "Научная литература", "Искусство и культура"],

  "игры": ["Игры", "Карточные игры", "Настольные игры", "Лото", "Домино и маджонг",
           "Игровые приставки", "Игральные карты", "Нарды", "Шашки", "Головоломки",
           "Настольные игры", "Досуг и творчество"],

  "кулинария": ["Кулинария", "Кулинарные книги", "Авторская посуда"],

  "путешествия": ["Путешествия", "Путешествия. Путеводители"],

  "искусство": ["Искусство", "Живопись", "Скульптура", "Фотография", "Танцы",
                "Театр", "Кино", "Писательство", "Поэзия", "Рукоделие", "Дизайн",
                "Марки", "Монеты", "Комиксы", "Антиквариат", "Картины и панно",
                "Искусство и культура"],

  "технологии": ["Технологии", "Компьютерные технологии", "Робототехника",
                 "Электронные и аудиокниги", "Цифровые товары", "Цифровые фоторамки",
                 "Интерактивные и электронные игрушки", "Компьютерные технологии"],

  "мода": ["Мода", "Аксессуары", "Одежда", "Обувь", "Сумки", "Ювелирные изделия"],

  "красота": ["Красота", "Косметика", "Парфюмерия", "Наборы для макияжа",
              "Наборы косметики", "Парфюмированный набор", "Косметические наборы",
              "Наборы средств для лица", "Наборы для гигиены"],

  "шоппинг": ["Шоппинг", "Аксессуары", "Одежда", "Обувь", "Сумки",
              "Ювелирные изделия", "Косметика", "Парфюмерия"],

  "автомобили": ["Автомобили", "Автомобильные аксессуары", "Автозапчасти"],

  "рыбалка": ["Рыбалка", "Рыболовные снасти", "Лодки"],

  "охота": ["Охота", "Охотничье снаряжение"],

  "финансы": ["Финансы", "Инвестиции и недвижимость", "Экономика",
              "Бухгалтерский учет и делопроизводство"],

  "образование": ["Образование", "Учебная литература", "Научно-популярная литература",
                  "Бизнес-образование и развитие карьеры", "Энциклопедии и справочники",
                  "Научная литература"],

  "здоровье": ["Здоровье", "Медицина", "Красота, здоровье и спорт", "Наборы для гигиены"],

  "саморазвитие": ["Саморазвитие", "Психология", "Эзотерика и спиритизм",
                   "Трансформационные психологические игры"],

  "домашние животные": ["Домашние животные", "Зоотовары"],

  "садоводство": ["Садоводство", "Цветы", "Инструменты и инвентарь", "Семена"],

  "фотография": ["Фотография", "Фотоаппараты", "Фотоальбомы", "Фотобутафория", "Цифровые фоторамки"],

  "танцы": ["Танцы"],

  "йога": ["Йога"],

  "волонтерство": ["Волонтерство"],

  "рисование": ["Рисование", "Краски, карандаши, кисти"],

  "писательство": ["Писательство", "Канцелярские товары"],

  "поэзия": ["Поэзия"],

  "рукоделие": ["Рукоделие", "Шитье", "Вязание", "Скрапбукинг", "Пэчворк и квилтинг",
                "Гобелены и стринг-арт", "Вышивание", "Валяние", "Создание украшений",
                "Декупаж", "Изготовление игрушек"],

  "дизайн": ["Дизайн", "Графический дизайн", "Веб-дизайн", "Дизайн интерьера"],

  "марки": ["Марки", "Филателистические аксессуары"],

  "монеты": ["Монеты", "Нумизматические аксессуары"],

  "комиксы": ["Комиксы"],

  "антиквариат": ["Антиквариат", "Антикварные газеты и журналы"],

  "игрушки": ["Игрушки", "Конструкторы", "Куклы и аксессуары",
              "Игрушечный транспорт", "Радиоуправляемые игрушки",
              "Мягкие игрушки", "Сюжетно-ролевые игрушки", "Игрушки для ванной",
              "Фигурки и аксессуары", "Игрушки для малышей",
              "Интерактивные и электронные игрушки", "Игрушечное оружие"],

  "видеоигры": ["Видеоигры", "Игровые приставки", "Компьютерные игры", "Мобильные игры"],

  "компьютерные игры": ["Компьютерные игры"],

  "мобильные игры": ["Мобильные игры"],

  "настольные игры": ["Настольные игры"],

  "азартные игры": ["Азартные игры", "Игральные карты", "Наборы для покера"]
}

### **Предобработка текста:** лемматизация и стемминг для полей "название", "описание" и "ключевые слова" (объединение "категории", "подкатегории" и "подподкатегории"), чтобы привести слова к базовой форме и улучшить поиск.


In [28]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [27]:
data_ = data.copy(deep=True)

In [29]:
data["текст"] = data[text_columns].apply(lambda x: ". ".join(x), axis=1)
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,1320591182,3.3,13,808,https://www.ozon.ru/product/proektor-dlya-smar...,"Электроника, Телевизоры и видеотехника, Телеви...",Проектор для смартфона 2-го поколения своими р...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,619939085,4.4,205,300,https://www.ozon.ru/product/proektor-90411567-...,"Электроника, Телевизоры и видеотехника, Телеви...","Проектор 90411567, белый. 3D увеличитель экран..."


In [32]:
data['текст'] = data['текст'].apply(preprocess_text)

In [33]:
data[['текст']].head(3)

,текст
0,проектор смартфон 2 го поколение свой рука под...
1,проектор 90411567 белый 3d увеличитель экран т...
2,проектор 90411553 3d увеличительный экран ваш ...


### **Векторизация данных:** TF-IDF и word2vec для преобразования текстовых данных ("название", "описание" и "ключевые слова") в числовые векторы.


#### TD-IDF

In [35]:
"""
# Создание TF-IDF векторайзера
vectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features

# Обучение векторайзера и преобразование текста в векторы
tfidf_vectors = vectorizer.fit_transform(data["текст"])

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Объединение с исходным DataFrame
data_tfidf = pd.concat([data, tfidf_df], axis=1)

data_tfidf.head(2)
"""

'\n# Создание TF-IDF векторайзера\nvectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features\n\n# Обучение векторайзера и преобразование текста в векторы\ntfidf_vectors = vectorizer.fit_transform(data["текст"])\n\n# Преобразование в DataFrame\ntfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())\n\n# Объединение с исходным DataFrame\ndata_tfidf = pd.concat([data, tfidf_df], axis=1)\n\ndata_tfidf.head(2)\n'

#### Word2Vec

In [36]:
"""
# Подготовка текстов для Word2Vec
text_corpus = [text.split() for text in data["текст"]]

# Обучение Word2Vec модели
model = Word2Vec(text_corpus, vector_size=10, window=5, min_count=5, workers=4)  # Настройте параметры

# Получение векторов для каждого текста
word2vec_vectors = []
for text in text_corpus:
    word_vectors = [model.wv[word] for word in text if word in model.wv]
    if word_vectors:
        text_vector = np.mean(word_vectors, axis=0)
    else:
        text_vector = np.zeros(model.vector_size)
    word2vec_vectors.append(text_vector)

# Преобразование в DataFrame
word2vec_df = pd.DataFrame(word2vec_vectors)

# Объединение с исходным DataFrame
data_word2vec = pd.concat([data, word2vec_df], axis=1)

data_word2vec.head(2)
"""

'\n# Подготовка текстов для Word2Vec\ntext_corpus = [text.split() for text in data["текст"]]\n\n# Обучение Word2Vec модели\nmodel = Word2Vec(text_corpus, vector_size=10, window=5, min_count=5, workers=4)  # Настройте параметры\n\n# Получение векторов для каждого текста\nword2vec_vectors = []\nfor text in text_corpus:\n    word_vectors = [model.wv[word] for word in text if word in model.wv]\n    if word_vectors:\n        text_vector = np.mean(word_vectors, axis=0)\n    else:\n        text_vector = np.zeros(model.vector_size)\n    word2vec_vectors.append(text_vector)\n\n# Преобразование в DataFrame\nword2vec_df = pd.DataFrame(word2vec_vectors)\n\n# Объединение с исходным DataFrame\ndata_word2vec = pd.concat([data, word2vec_df], axis=1)\n\ndata_word2vec.head(2)\n'

### **Нормализация числовых данных:** Нормализовация данные о цене и количестве отзывов, чтобы они имели одинаковый масштаб.


In [91]:
# normalize_column = [column for column in data.columns if data[column].dtype != 'object' and column != 'код товара']
# data[normalize_column].head(2)

,оценка,количество отзывов,цена
0,3.3,13,808
1,4.4,205,300


In [92]:
# scaler = MinMaxScaler().fit(data[normalize_column])

# data_normalize = scaler.transform(data[normalize_column])
# data[normalize_column] = pd.DataFrame(data_normalize, columns = normalize_column)

# data[normalize_column].head(2)

,оценка,количество отзывов,цена
0,0.66,0.000142,0.000457
1,0.88,0.002238,0.000169


### **??? Кодирование категориальных данных:** Использовать one-hot encoding для категориальных данных, таких как пол и интересы.

## **2.  Выбор модели:**

* **Контент-базированная фильтрация:**
    * Вычислить сходство между векторами запроса пользователя (пол, возраст, интересы) и векторами товаров.
    * Для этого можно использовать косинусную меру или евклидово расстояние.
    * Рекомендуйте товары с наибольшим сходством.

* **Коллаборативная фильтрация:**
    * Если доступны данные о прошлых покупках или оценках пользователей, можно использовать алгоритмы SVD или ALS для выявления скрытых факторов, влияющих на выбор подарков.
    * На основе этих факторов можно прогнозировать, какие товары понравятся пользователю с заданным профилем.


In [44]:
from sklearn.metrics.pairwise import linear_kernel

def filter_products(data, query, interests, price_range):
    # Пример фильтрации по ценовому диапазону
    filtered_data = data[(data['цена'] >= price_range[0]) & (data['цена'] <= price_range[1])]

    # Пример фильтрации по интересам
    tfidf = TfidfVectorizer(stop_words=russian_stopwords)
    tfidf_matrix = tfidf.fit_transform(filtered_data['текст'])
    query = query + ' ' + ' '.join(interests)
    query_vec = tfidf.transform([query])

    filtered_data['similarity'] = linear_kernel(query_vec, tfidf_matrix).flatten()
    filtered_data = filtered_data.sort_values(by=['similarity', 'оценка', 'количество отзывов'], ascending=False)

    return filtered_data.head(10)

# Пример запроса пользователя
query = 'подарок маме на день рождения 30 лет'
interests = ['вязание', 'садоводство', 'здоровье']
price_range = (500, 2000)

# Получение рекомендаций
filter_products(data, query, interests, price_range)[['название', 'ключевые слова',  'цена', 'ссылка', 'код товара']]

<ipython-input-44-3102ac78f26a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['similarity'] = linear_kernel(query_vec, tfidf_matrix).flatten()


,название,ключевые слова,цена,ссылка,код товара
18048,"Набор для вязания крючком с бисером ""Колье и б...","Хобби и творчество, Наборы для мастер-классов,...",1570,https://www.ozon.ru/product/nabor-dlya-vyazani...,963506423
18060,"Набор для вязания крючком с бисером ""Колье и б...","Хобби и творчество, Наборы для мастер-классов,...",1490,https://www.ozon.ru/product/nabor-dlya-vyazani...,950557268
18050,"Набор для вязания крючком с бисером ""Колье (бр...","Хобби и творчество, Наборы для мастер-классов,...",1346,https://www.ozon.ru/product/nabor-dlya-vyazani...,1254982050
10330,"Набор крючков для вязания в органайзере, 16 шт...","Хобби и творчество, Рукоделие, Рукоделие",1288,https://www.ozon.ru/product/nabor-kryuchkov-dl...,825802942
10295,Спицы для вязания круговые 80 см. Набор деревя...,"Хобби и творчество, Рукоделие, Рукоделие",669,https://www.ozon.ru/product/spitsy-dlya-vyazan...,893883286
10362,Трубчатая Фантазийная пряжа для вязания руками...,"Хобби и творчество, Рукоделие, Рукоделие",1222,https://www.ozon.ru/product/trubchataya-fantaz...,1476281485
10338,Крючок для вязания Clover 2 мм,"Хобби и творчество, Рукоделие, Рукоделие",1043,https://www.ozon.ru/product/kryuchok-dlya-vyaz...,856795165
10357,Набор для вязания крючком игрушки - Куколка По...,"Хобби и творчество, Рукоделие, Рукоделие",1215,https://www.ozon.ru/product/nabor-dlya-vyazani...,1497947910
10511,"Набор для вязания игрушки крючком - Мишка ""Mar...","Хобби и творчество, Рукоделие, Рукоделие",688,https://www.ozon.ru/product/nabor-dlya-vyazani...,1497877819
10331,"Набор для вязания Почита человек-бензопила ""cr...","Хобби и творчество, Рукоделие, Рукоделие",1278,https://www.ozon.ru/product/nabor-dlya-vyazani...,1420562368


## **3.  Гибридный подход:** (пока этого не будет)

* Комбинируйте контент-базированную и коллаборативную фильтрацию для большей точности рекомендаций.



## **4. Ранжирование и фильтрация:**

* **Ранжирование:** Отсортируйте рекомендации по убыванию сходства или прогнозируемого рейтинга.
* **Фильтрация:** Примените фильтры по цене, категории, подкатегории и т.д., чтобы сузить список рекомендаций в соответствии с предпочтениями пользователя.



## **5.  Дополнительные функции:**

* **Персонализация:** Используйте информацию о прошлых взаимодействиях пользователя с системой для повышения релевантности рекомендаций.
* **Разнообразие:** Стремитесь к разнообразию в рекомендациях, избегая предлагать слишком похожие товары.
* **Объяснение:** Отобразите пользователю, почему ему были рекомендованы те или иные товары.